# A*

In [1]:
from queue import PriorityQueue
import numpy as np
from enum import Enum

In [2]:
class Action(Enum):
    """
    An action is represented by a 3 element tuple.
    
    The first 2 values are the delta of the action relative
    to the current grid position. The third and final value
    is the cost of performing the action.
    """
    UP = (-1, 0, 1)
    URIGHT = (-1, 1, 3*np.math.sqrt(2))
    RIGHT = (0, 1, 1)
    DRIGHT = (1, 1, 3*np.math.sqrt(2))
    DOWN = (1, 0, 1)
    DLEFT = (1, -1, 3*np.math.sqrt(2))
    LEFT = (0, -1, 1)
    ULEFT = (-1, -1, 3*np.math.sqrt(2))
    
    def __str__(self):
        if self == self.UP:
            return '↑'
        elif self == self.URIGHT:
            return '↗'
        elif self == self.RIGHT:
            return '→'
        elif self == self.DRIGHT:
            return '↘'
        elif self == self.DOWN:
            return '↓'
        elif self == self.DLEFT:
            return '↙'
        elif self == self.LEFT:
            return '←'
        elif self == self.ULEFT:
            return '↖'
        
    @property
    def cost(self):
        return self.value[2]
    
    @property
    def delta(self):
        return (self.value[0], self.value[1])

def valid_actions(grid, current_node):
    """
    Returns a list of valid actions given a grid and current node.
    """
    # Select all actions as valid.
    valid = [a for a in Action]
    n, m = grid.shape[0] - 1, grid.shape[1] - 1
    x, y = current_node
    
    for a in Action:
        new_x = x + a.delta[0]
        new_y = y + a.delta[1]
        # Check if the node is off the grid
        if new_x < 0 or new_x > n or new_y < 0 or new_y > m:
            valid.remove(a)
        # Check if the node is an obstacle
        elif grid[new_x, new_y] == 1:
            valid.remove(a)
        
    return valid

def visualize_path(grid, path, start):
    sgrid = np.zeros(np.shape(grid), dtype=np.str)
    sgrid[:] = ' '
    sgrid[grid[:] == 1] = 'O'
    
    pos = start
    
    for a in path:
        da = a.value
        sgrid[pos[0], pos[1]] = str(a)
        pos = (pos[0] + da[0], pos[1] + da[1])
    sgrid[pos[0], pos[1]] = 'G'
    sgrid[start[0], start[1]] = 'S'  
    return sgrid

## Heuristics
The heuristic function determines the $h()$ value for each cell based on the goal cell and the method chosen to determine it. The heuristic value can be the Euclidean distance between these cells $h= \left((x_i-x_{goal})^2+(y_i-y_{goal})^2\right)^{1/2}$ or the "Manhattan distance", which is the minimum number of moves required to reach the goal from the assigned cell $h = ||x_i-x_{goal}|| + ||y_i-y_{goal}||$. For this exercise you could use either, or something else which is *admissible* and *consistent*.

The input variables include
* **```position```** the coordinates of the cell for which you would like to determine the heuristic value.
* **```goal_position```** the coordinates of the goal cell

In [3]:
# TODO: implement a heuristic function. This may be one of the
# functions described above or feel free to think of something
# else.
def heuristic(position, goal_position):
    # Use euclidean distance as the heuristic for now
    h = np.math.sqrt((goal_position[0] - position[0])**2 +
                     (goal_position[1] - position[1])**2)
    return h

## A* search

A* search is an extension of the cost search you implemented. A heuristic function is used in addition to the cost penalty. Thus if the setup is:

* $c$ is the current cost
* $g$ is the cost function
* $h$ is the heuristic function

Then the new cost is $c_{new} = c + g() + h()$.

The difference between $g$ and $h$ is that $g$ models the cost of performing actions, irrespective of the environment, while $h$ models the cost based on the environment, i.e., the distance to the goal.

You know what comes next, turn the `TODOs` into `DONEs` :)

In [8]:
def a_star(grid, h, start, goal):
    # Initialize search variables.
    queue = PriorityQueue()
    queue.put((0, start))
    visited = set(start)
    branch = {}
    found = False
    
    while not queue.empty():
        #print(queue.queue)
        item = queue.get()
        current_node = item[1]
        if current_node == start:
            current_cost = 0.0
        else:              
            current_cost = branch[current_node][0]

        if current_node == goal:        
            print('Found a path.')
            found = True
            break
        else:
            for action in valid_actions(grid, current_node):
                # get the tuple representation
                da = action.delta
                next_node = (current_node[0] + da[0], 
                             current_node[1] + da[1])
                # Calculate branch cost (action.cost + g)
                branch_cost = action.cost + current_cost
                # Calculate queue cost (action.cost + g + h)
                queue_cost = branch_cost + heuristic(next_node, goal)
                
                #print("Adding", (queue_cost, next_node))
                if next_node not in visited:                
                    visited.add(next_node)               
                    branch[next_node] = (branch_cost, current_node, action)
                    queue.put((queue_cost, next_node))    
                #else:
                #    print("  ... already in visited set")
          
    path = []
    path_cost = 0
    if found:
        # retrace steps
        n = goal
        path_cost = branch[n][0]
        while branch[n][1] != start:
            path.append(branch[n][2])
            n = branch[n][1]
        path.append(branch[n][2])
    else:
        print('**********************')
        print('Failed to find a path!')
        print('**********************')
        
    return path[::-1], path_cost

In [5]:
start = (0, 0)
goal = (4, 4)

grid = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 0],
])

In [9]:
path, cost = a_star(grid, heuristic, start, goal)
print(path, cost)

Found a path.
[<Action.DRIGHT: (1, 1, 4.242640687119286)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.DRIGHT: (1, 1, 4.242640687119286)>, <Action.DRIGHT: (1, 1, 4.242640687119286)>, <Action.DLEFT: (1, -1, 4.242640687119286)>] 18.970562748477143


In [7]:
# S -> start, G -> goal, O -> obstacle
visualize_path(grid, path, start)

array([['S', 'O', ' ', ' ', ' ', ' '],
       [' ', '→', '→', '↘', ' ', ' '],
       [' ', 'O', ' ', ' ', '↘', ' '],
       [' ', ' ', ' ', 'O', 'O', '↙'],
       [' ', ' ', ' ', 'O', 'G', ' ']], dtype='<U1')

[Solution](/notebooks/A-Star-Solution.ipynb)